In [1]:
## all imports
from IPython.display import HTML
import chardet
import io
import numpy as np
np.seterr(divide='ignore', invalid='ignore')
import os
import urllib.request
from urllib.request import urlopen
from bs4 import BeautifulSoup
from time import sleep
import requests
import operator
import socket
import _pickle
import math
import re # regular expressions

from pandas import Series
import pandas as pd
from pandas import DataFrame

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (20, 10)
import json

import seaborn as sns
sns.set_context("talk")
sns.set_style("white")

from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time
from datetime import datetime

#from secret import *

### FUNCTION TO CREATE URL

In [2]:
def find_url(row):
    Base_Link_01= "https://en.wikipedia.org/wiki/"
    Base_Link_02= "_AFL_season"
    Variable= row
    link = Base_Link_01 + Variable + Base_Link_02
    return (link)

### FUNCTION TO ENSURE ALL MONTH SYNTAX IS CONSISTENT

In [3]:
def correct_month (x):
    first_space_index = x.index(" ")
    day = x[:first_space_index]
    #print (day)
    time = x[first_space_index:].strip()
    space_index = time.index(" ")
    hours = time[space_index:].strip()
    split_month = x.split(" ")[1]
    #print (split_month)
    
    months_tocheck = ["January", "Febuary ", "March", "April", "May", "June", "July", "August", "September", "October", "Novemeber", "December"]
    
    #if any (split_month for month in months_tocheck):
    if split_month in months_tocheck:
        #print ("it is a correct month")
        return (x)
    
       
    else:
        month_dict = {
        "Mar" : "March",
        "Apr" : "April",
        "May" : "May",
        "June" : "June",
        "July" : "July",
        "Aug" : "August",
        "Sep" : "September",
        "Oct" : "October",
        }
        #print (month_dict[split_month])
        #print ("is not a correct minth")
        return (day + " " + month_dict[split_month] + " " + hours)


        


### FUNCTION TO RETURN DATE

In [4]:
def the_date (x):
    first_space_index = x.index(" ")
    day = x[:first_space_index]
    return (day)

### FUNCTION TO RETURN MONTH

In [5]:
def the_month (x):
    first_space_index = x.index(" ")
    day = x[:first_space_index]
    #print (day)
    time = x[first_space_index:].strip()
    space_index = time.index(" ")
    hours = time[space_index:].strip()
    split_month = x.split(" ")[1]
    return (split_month)
    #print (split_month)

### FUNCTION TO RETURN TIME (HOURS)

In [6]:
def the_time (x):
    first_space_index = x.index(" ")
    day = x[:first_space_index]
    #print (day)
    time = x[first_space_index:].strip()
    space_index = time.index(" ")
    hours = time[space_index:].strip()
    split_month = x.split(" ")[1]
    return (hours[1:-1])
    #print (split_month)

### FUNCTION TO RETURN DAY IF IT EXISTS

In [19]:
def if_dayexists (x):
    ## we know if it has a day it will have a comma
    #x = "Thursday, 14 June (5:40 pm)"
    count = x.count(",")
    if count >0:
        date = x.split(",")[0].strip()
        #print (date)
        #first_colon_index = x.index(",")
        #print (first_colon_index)
        #date = x[first_colon_index+1:].strip()
        return (date)
    else: 
        date = x
        return (x, "No Day Data Provided")
    


### FUNCTION TO CREATE STRIPTIME + DATETIME

In [8]:
def structured_times (x):

    struct_time = time.strptime(x, "%Y %d %B (%I:%M %p)")
    dt = datetime.fromtimestamp(time.mktime(struct_time))
    return (dt)

### FUNCTION TO CONVERT TO CONSISTENT DATE SYNTAX (NO PF1 AT FRONT)

In [9]:
def date_conversion (x):
    #x = "PF1: Friday, 25 September (6:20 pm)"
    #x = "Thursday, 14 June (5:40 pm)"
    count = x.count(":")
    if count >1:
        first_colon_index = x.index(":")
        date = x[first_colon_index+1:].strip()
        return (date)
    else: 
        date = x
        return (date)

### REMOVE DAY FOR STRPTIME CONSISTENCY

In [10]:
def day_conversion (x):
    ## we know if it has a day it will have a comma
    #x = "Thursday, 14 June (5:40 pm)"
    count = x.count(",")
    if count >0:
        date = x.split(",")[1].strip()
        #print (date)
        #first_colon_index = x.index(",")
        #print (first_colon_index)
        #date = x[first_colon_index+1:].strip()
        return (date)
    else: 
        date = x
        return (date)
    
    


### FUNCTION TO SORT DATA INTO DICTIONARY FOR EACH MATCH

In [279]:
def match_function_typical(match, this_round, this_year):

    try:

        namestocheck_good = ["Round", "Qualifying ", "Elimination", "Semi", "Preliminary"]
        namestocheck_bad = ["NAB"]
        if any (name in this_round for name in namestocheck_good):
            #print ("entered inside" + this_round)

            ### WE ONLY WANT TO LOOK WHEN THE ROUND NAME INCLUDES THAT ABOVE
            test = match.find_all('td')

            
            
            ### LOOKING AT THE DATES
            
            
            # FUNCTION TAKE OFF FRONT PREFIX
            time_date= date_conversion(test[0].string)
            #print (time_date)
            
            ### FUNCTION TO ASSIGN DAY IF IT EXISTS (BEFORE WE STRIP IT OFF FOR CLARITY AS NOT NEEDED)
            day = if_dayexists (time_date)
            #print (day)
            
            ### FUNCTION TO TAKE OFF THE DAY AT THE START (AS NOT CONSISTENT)
            convert_time = day_conversion(time_date)
            
            #print ("this far 2")
            ### FUNCTION TO ENSURE CONSISTENT MONTH SYNTAX
            correct_time_syn = correct_month (convert_time)
            
            #print (correct_time_syn)
            ### FUNCTION TO ASSIGN DATE, MONTH AND TIME
            date = the_date(correct_time_syn)
            month = the_month(correct_time_syn)
            time = the_time(correct_time_syn)
            
            
            ### ADDS THE CURRENT YEAR
            convert_time_year = (str(this_year)+ " " + correct_time_syn)
            


            ### FUNCTION THAT CONVERTS TO TIME STRUCTURE
            struct_time = structured_times(convert_time_year)

            #print ("this far 2")
            ### LOOK AT MATCH DATA

            #print (struct_time)
            home_team = test[1].find('a').string
            #print (home_team)
            home_score = (''.join(text for text in test[1].find_all(text=True) if text.parent.name != "a")).strip()
            #print (home_score)
            home_goals= home_score.split('.')[0]
            #print ("point of failure")
            home_behinds= home_score.split('.')[1].split('(')[0]

            home_total = (home_score.split('.')[1].split('(')[1])[:-1]

            #print (home_team)
            #print (home_goals)
            #print (home_behinds)
            #print (home_total)


            #print ("fail")
            away_team = test[3].find('a').string
            away_score = (''.join(text for text in test[3].find_all(text=True) if text.parent.name != "a")).strip()
            away_goals= away_score.split('.')[0]
            away_behinds= away_score.split('.')[1].split('(')[0]
            away_total = (away_score.split('.')[1].split('(')[1])[:-1]
            #print (away_team)
            #print (away_goals)
            #print (away_behinds)
            #print (away_total)

            #print ("fail")

            venue = test[4].find('a').string
            venue_crowd = (''.join(text for text in test[4].find_all(text=True) if text.parent.name != "a")).strip()
            venue_crowd_= (venue_crowd.split(':')[1])[:-1].strip()
            final_crowd = venue_crowd_.replace(",", "")
            #print (venue)
            #print (final_crowd)


            match_report_url = test[5].find('a')['href']
            #print (match_report_url)


            ind_match_dict= {
                 "Day": day,
                 "Date": date,
                 "Month": month,
                 "Year": this_year,
                 "Round": this_round,         
                 "Time": time,
                 "Structured Data": struct_time,
                 "Home Team": home_team,
                 "Home Goals": home_goals,
                 "Home Behinds": home_behinds,
                 "Home Total": home_total,
                 "Away Team": away_team,
                 "Away Goals": away_goals,
                 "Away Behinds": away_behinds,
                 "Away Total": away_total,
                 "Venue": venue,
                 "Crowd": final_crowd,
                 "Match Link": match_report_url}




            return (ind_match_dict)
        
        else:
            print ("no name in scope", this_round)
    
    except Exception as e:
        print("this row does not have match data", e)
        #print ("/t",convert_time_year)
        #return False



In [323]:
def match_function_grandf(match, this_round, this_year):

    try:

        namestocheck_good = ["Round", "Qualifying ", "Elimination", "Semi", "Preliminary", "Grand"]
        namestocheck_bad = ["NAB"]
        if any (name in this_round for name in namestocheck_good):
            #print ("entered inside" + this_round)

            ### WE ONLY WANT TO LOOK WHEN THE ROUND NAME INCLUDES THAT ABOVE
            test = match.find_all('td')

            
            
            ### LOOKING AT THE DATES
            
            
            # FUNCTION TAKE OFF FRONT PREFIX
            time_date= date_conversion(test[0].string)
            #print (time_date)
            
            ### FUNCTION TO ASSIGN DAY IF IT EXISTS (BEFORE WE STRIP IT OFF FOR CLARITY AS NOT NEEDED)
            day = if_dayexists (time_date)
            #print (day)
            
            ### FUNCTION TO TAKE OFF THE DAY AT THE START (AS NOT CONSISTENT)
            convert_time = day_conversion(time_date)
            
            #print ("this far 2")
            ### FUNCTION TO ENSURE CONSISTENT MONTH SYNTAX
            correct_time_syn = correct_month (convert_time)
            
            #print (correct_time_syn)
            ### FUNCTION TO ASSIGN DATE, MONTH AND TIME
            date = the_date(correct_time_syn)
            month = the_month(correct_time_syn)
            time = the_time(correct_time_syn)
            
            
            ### ADDS THE CURRENT YEAR
            convert_time_year = (str(this_year)+ " " + correct_time_syn)
            


            ### FUNCTION THAT CONVERTS TO TIME STRUCTURE
            struct_time = structured_times(convert_time_year)


            home_team = test[1].find('a').string
            away_team = test[3].find('a').string
            
            venue = test[4].find('a').string
            venue_crowd = (''.join(text for text in test[4].find_all(text=True) if text.parent.name != "a")).strip()
            venue_crowd_= (venue_crowd.split(':')[1])[:-1].strip()
            final_crowd = venue_crowd_.replace(",", "")



            match_report_url = test[5].find('a')['href']
            #print (match_report_url)
            
            
            home_score = (''.join(text for text in test[1].find_all(text=True) if text.parent.name != "a")).strip()
            print (home_score)
            home_goals= home_score.split('.')[0]
            print (home_goals)
            
            if not home_goals:
                print ("no home goals here")

                ind_match_dict= {
                     "Day": day,
                     "Date": date,
                     "Month": month,
                     "Year": this_year,
                     "Round": this_round,         
                     "Time": time,
                     "Structured Data": struct_time,
                     "Home Team": home_team,
                     "Home Goals": 'TBC',
                     "Home Behinds": "TBC",
                     "Home Total": "TBC",
                     "Away Team": away_team,
                     "Away Goals": "TBC",
                     "Away Behinds": "TBC",
                     "Away Total": "TBC",
                     "Venue": venue,
                     "Crowd": final_crowd,
                     "Match Link": match_report_url}
            
            else: 
                
                print ('we have goals provided')
                ind_match_dict= {
                     "Day": day,
                     "Date": date,
                     "Month": month,
                     "Year": this_year,
                     "Round": this_round,         
                     "Time": time,
                     "Structured Data": struct_time,
                     "Home Team": home_team,
                     "Home Goals": home_goals,
                     "Home Behinds": "TBC",
                     "Home Total": "TBC",
                     "Away Team": away_team,
                     "Away Goals": "TBC",
                     "Away Behinds": "TBC",
                     "Away Total": "TBC",
                     "Venue": venue,
                     "Crowd": final_crowd,
                     "Match Link": match_report_url}

            return (ind_match_dict)
        
        else:
            print ("no name in scope", this_round)
    
    except Exception as e:
        print("this row does not have match data", e)
        #print ("/t",convert_time_year)
        #return False



### FUNCTION TO RETURN A LIST OF DATA FOR EACH ROUND

In [293]:
def round_function(round_number, year_data,current_year):    
    games= year_data[round_number].find_all('tr')
    #print (games)
    ## Find the round header
    round_head = (''.join(text for text in games[1].find_all(text=True) if text.parent.name != "a")).strip()
    #print (round_head)
    #if isinstance(round_head,str) == False:
    ## Check is string is empty- empty strings are falsy- so we are asking if not false- double negative- not false = true
    if not round_head:
        round_head = games[1].find('a').string

    this_round = []
    
 

    if round_head != "Grand Final" and round_head != "Grand Final Replay":
        #print (round_head)
        #print (round_head)
        round_matchday = games[2:-1]
        


        for match in round_matchday:
            #print (match)
            result= match_function_typical(match, round_head, current_year)
            if result:
                this_round.append(result)
        return (this_round)


    else:  
        #print (round_head)
        #return("grand final")
        #round_matchday_final = games[0:]
        #print (round_matchday_final)
        result= match_function_grandf(games[2],round_head, current_year)
        if result:
            this_round.append(result)
        return (this_round)


### RETURN EACH ROUND FOR EACH YEAR FROM FUNCTIONS INTO MEGA LIST

In [325]:
mega_list = []
# 2012- NO GRAND FINAL
# 2011- NO GRAND FINAL
# 2010- NO GRAND FINAL
# 2009- NO GRAND FINAL
test_year = 2010

for year in list(range(test_year,test_year + 1)):
#for year in list(range(2009,2018,1)):
    current_year = str(year)
    url = find_url(current_year)
    req = requests.get(url)
    soup = BeautifulSoup(req.text, 'html.parser')
    ### FIND ALL OF THE TABLES THAT MATCH THE ROUND CRITERIA
    year_data = soup.find("div", {"id": "mw-content-text"}).find_all('table', {"style": "width: 100%; background-color: #f1f5fc; border: 2px solid #D0E5F5"})
    ### COUNT LENGTH OF ROUND TO ASSIGN TO LOOP
    round_range = len(year_data)
    

    
    #print (current_year)
    for current_round in range(round_range):

        this_round = round_function(current_round, year_data, current_year)
        mega_list.extend(this_round)
        #print (this_round)

    #print (current_year)
print (mega_list)




no home goals here


no home goals here
[{'Day': 'Thursday', 'Date': '25', 'Month': 'March', 'Year': '2010', 'Round': 'Round 1 (season launch)', 'Time': '7:10\xa0pm', 'Structured Data': datetime.datetime(2010, 3, 25, 19, 10), 'Home Team': 'Richmond', 'Home Goals': '9', 'Home Behinds': '10\xa0', 'Home Total': '64', 'Away Team': 'Carlton', 'Away Goals': '18', 'Away Behinds': '12\xa0', 'Away Total': '120', 'Venue': 'MCG', 'Crowd': '72010', 'Match Link': 'http://www.afl.com.au/news/newsarticle/tabid/208/newsid/91077/default.aspx'}, {'Day': 'Friday', 'Date': '26', 'Month': 'March', 'Year': '2010', 'Round': 'Round 1 (season launch)', 'Time': '7:40\xa0pm', 'Structured Data': datetime.datetime(2010, 3, 26, 19, 40), 'Home Team': 'Geelong', 'Home Goals': '19', 'Home Behinds': '11\xa0', 'Home Total': '125', 'Away Team': 'Essendon', 'Away Goals': '13', 'Away Behinds': '16\xa0', 'Away Total': '94', 'Venue': 'MCG', 'Crowd': '57772', 'Match Link': 'http://www.afl.com.au/news/newsarticle/tabid/208/n

### TURN MEGALIST INTO DATAFRAME

In [257]:
df = pd.DataFrame(mega_list)
df.head(400)
#df.shape

,Away Behinds,Away Goals,Away Team,Away Total,Crowd,Date,Day,Home Behinds,Home Goals,Home Team,Home Total,Match Link,Month,Round,Structured Data,Time,Venue,Year
0,6,13,Richmond,84,60654,24,Thursday,20,14,Carlton,104,https://web.archive.org/web/20110416022924/htt...,March,Round 1,2011-03-24 19:40:00,7:40 pm,MCG,2011
1,11,6,St Kilda,47,42869,25,Friday,12,6,Geelong,48,http://www.afl.com.au/news/newsarticle/tabid/2...,March,Round 1,2011-03-25 19:40:00,7:40 pm,MCG,2011
2,8,12,Port Adelaide,80,35773,26,Saturday,11,24,Collingwood,155,http://www.afl.com.au/news/newsarticle/tabid/2...,March,Round 1,2011-03-26 14:10:00,2:10 pm,Etihad Stadium,2011
3,13,12,Hawthorn,85,42536,26,Saturday,9,16,Adelaide,105,https://web.archive.org/web/20121007172516/htt...,March,Round 1,2011-03-26 19:10:00,7:10 pm,AAMI Stadium,2011
4,16,13,Fremantle,94,23373,26,Saturday,8,14,Brisbane Lions,92,https://web.archive.org/web/20121007172527/htt...,March,Round 1,2011-03-26 19:10:00,7:10 pm,The Gabba,2011
5,10,8,Western Bulldogs,58,42617,27,Sunday,17,16,Essendon,113,http://www.afl.com.au/news/newsarticle/tabid/2...,March,Round 1,2011-03-27 13:10:00,1:10 pm,Etihad Stadium,2011
6,18,11,Sydney,84,33951,27,Sunday,18,11,Melbourne,84,http://www.afl.com.au/news/newsarticle/tabid/2...,March,Round 1,2011-03-27 14:10:00,2:10 pm,MCG,2011
7,10,13,North Melbourne,88,35878,27,Sunday,14,13,West Coast,92,http://www.afl.com.au/news/newsarticle/tabid/2...,March,Round 1,2011-03-27 16:10:00,4:10 pm,Patersons Stadium,2011
8,11,14,Richmond,95,41465,1,Friday,17,13,St Kilda,95,https://web.archive.org/web/20110404113541/htt...,April,Round 2,2011-04-01 19:40:00,7:40 pm,MCG,2011
9,17,21,Collingwood,143,40578,2,Saturday,14,7,North Melbourne,56,http://www.afl.com.au/news/newsarticle/tabid/2...,April,Round 2,2011-04-02 14:10:00,2:10 pm,Etihad Stadium,2011
